In [ ]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import pandas as pd 
from sklearn.metrics import mean_absolute_percentage_error
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

from IPython.display import SVG
IPythonConsole.ipython_useSVG=True
%matplotlib inline 

# Проект по предсказанию характеристики связывания малых молекул с лекарственной мишенью. (Связываемость с рецепторами дофамина)

https://www.kaggle.com/datasets/bhawakshi/pec50-prediction-dopamine-receptor?resource=download&select=enriched_dopamine_ec50.csv  
Отсюда был взял датасет

Состоит из четрых частей: 
1) Построение регресионной модели основанной на отпечатках моргана для предсказания значений pEC50
2) Построение регресионной модели основанной на дескрипторах RDKit для того же предсказания (без и с работой с полученными дескрипторами)
3) Построение классификатора по типам допаминовых рецепторов (Target Name) на отпечатках моргана
4) Построение классификатора основанного на дескрипторах RDKit 


# Подготовка датасета 

Экспериментальные данные: Значения EC50 (нМ), значения pEC50 (-lg(EC50)).

Тип анализа(Assay Type) и название мишени (Target Name): Воспроизводимость и подтип рецептора - D1, D2, D3, D4 и D5.

LogP (липофильность): Указывает на гидрофобность.

H_Donors и H_Acceptors: Показатели способности к водородным связям.

TPSA (топологическая площадь полярной поверхности): Важна для биодоступности.

Количество колец (Ring_Count) и количество вращающихся связей (Rotatable_Bonds): Показатели сложности молекулы.

EC50 - или полумаксимальная эффективная концентрация, означает концентрацию лиганда, которая вызывает эффект, равный половине максимального возможного для данного лиганда после истечения некоторого промежутка времени. 

In [ ]:
df = pd.read_csv('enriched_dopamine_ec50.csv')
df = df.drop(['ChEMBL ID'],axis=1)
df

In [ ]:
df.describe()

Посмотрим на распределение потенциального таргета:

In [ ]:
df['pEC50'].value_counts()

In [ ]:
ax = plt.hist(df['pEC50'], bins=100)

In [ ]:
import pylab 
import scipy.stats as stats
stats.probplot(df['pEC50'], dist="norm", plot=pylab)
pylab.show()

Как видно из гистограммы выше, значения pEC50 = 5 и 4 являются довольно очевидными выбросами, что довольно просто объясняется тем, что в статьях значение 5 часто выбирают в качестве порога, то есть приближения, оно не отражает действительное значение pEC50
qq plot Также показывает, что наш будущих таргет распрелен не нормально и есть выбросы

In [ ]:
df_u = df[(df['pEC50']!=5) & (df['pEC50']!=4)]
df_u

In [ ]:
ax = plt.hist(df_u['pEC50'], bins=100)

Дополнитель проведем квартильную фильтрацию

In [ ]:
Q1 = df_u['pEC50'].quantile(0.2)
Q3 = df_u['pEC50'].quantile(0.8)
IQR = Q3 - Q1
filtered_df =  df_u[~((df_u['pEC50'] < (Q1 - 1.5 * IQR)) |(df_u['pEC50'] > (Q3 + 1.5 * IQR)))]
filtered_df

Построим qq plot чтобы сравнить наше распределение с нормальным более наглядно:

In [ ]:
import pylab 
import scipy.stats as stats
stats.probplot(filtered_df['pEC50'], dist="norm", plot=pylab)
pylab.show()

Распределение теперь выглядит хорошо, можно работать дальше

In [ ]:
df_work = filtered_df[['SMILES','EC50 (nM)','pEC50']]
df_work

Посмотрим на распределение таргета в случае -лог() и простой формы:

In [ ]:
ax = plt.hist(df_work['EC50 (nM)'], bins=50)

In [ ]:
ax = plt.hist(df_work['pEC50'], bins=20)

Очевидно, что распределение таргета в случае отрицательного логаримфа гораздо ближе к нормальному, будем предсказывать именно его (переменную pEC50)

In [ ]:
df_work = df_work.drop('EC50 (nM)',axis=1)
df_work

# Часть 1 

# Добавление отпечатков в рабочий сет

На данный момент стоит выбрать оптимальное значение радиуса и битности:
Для нашей задачи подойдет перебор радиусов в диапазоне от 1 до 3, и битности  512 1024 и 2048.

На всякий случай канонизируем смайлз

In [ ]:
df_work['SMILES'] = df_work['SMILES'].apply(lambda x: Chem.CanonSmiles(x))

Добавим столбец с мол-файлами

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df_work, smilesCol='SMILES', molCol='Molecule')

In [ ]:
df_work

Добавим разные  отпечатки пальцев

In [ ]:
def fpgen(x,n,size):
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=n,fpSize=size,includeChirality=True)
    e = mfpgen.GetFingerprint(x)#получаем FP
    return (e) 

In [ ]:
r = [1,2,3]
bit = [512,1024,2048]
for i in r:
    for b in bit:
        df_work[f'fp{i},{b}'] = df_work['Molecule'].apply(lambda x: fpgen(x,i,b))
df_work

Я выберу оптимальное значение радуиса и битности путем использования RandomForestRegressor как показательной модели, после чего уже проведу побор гиперпараметров 

Разбиваем сет на трейн и тест с самого начала

In [ ]:
X = df_work.drop(['SMILES','pEC50','Molecule'],axis=1)
y = df_work['pEC50']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=13)

In [ ]:
params = {
    "n_estimators": 100,
    "max_depth": 15,
    "min_samples_split": 2,'min_samples_leaf': 1 
,'random_state':13}

Здесь я включил режим программиста и решил написать код который будет перебирать все типы отпечатков и смотреть эффективность регресионной модели добавляя в конечный список значение r2 и название столбца (конечно я мог 9 раз запустить код, но это было бы не так круто :)

In [ ]:
col = X_train.columns
r2_l = []
for c in col:
    e = np.ones(shape=(512,))
    e2= np.ones(shape=(1024,))
    e3= np.ones(shape=(2048,))
    d = np.ones(shape=(512,))
    d2= np.ones(shape=(1024,))
    d3= np.ones(shape=(2048,))    
    
    for fp in X_train[c]:
        if len(np.array(fp)) == 512:
            e = np.vstack((e,np.array(fp)))
        elif len(np.array(fp)) == 1024:
            e2 = np.vstack((e2,np.array(fp)))
        elif len(np.array(fp)) == 2048:
            e3 = np.vstack((e3,np.array(fp)))
    
    if  e.shape != (512,):
        e = np.delete(e,0,0)
        Train = e
    elif e2.shape != (1024,):
        e2 = np.delete(e2,0,0)
        Train = e2
    elif e3.shape != (2048,):
        e3 = np.delete(e3,0,0)
        Train = e3

    for fp in X_test[c]:
        if len(np.array(fp)) == 512:
            d = np.vstack((d,np.array(fp)))
        elif len(np.array(fp)) == 1024:
            d2 = np.vstack((d2,np.array(fp)))
        else:
            d3 = np.vstack((d3,np.array(fp)))
    
    if  d.shape != (512,):
        d = np.delete(d,0,0)        
        Test = d
    elif d2.shape != (1024,):
        d2 = np.delete(d2,0,0)         
        Test = d2
    elif d3.shape != (2048,):
        d3 = np.delete(d3,0,0)         
        Test = d3 
    reg = RandomForestRegressor(**params).fit(Train, y_train)
    r2 = r2_score(y_test, reg.predict(Test))
    r2_l.append([c,r2])
    print(r2_l[-1]) 


Из выведенной выше таблицы можно сделать вывод, что оптимальный выбор - это радиус = 2, и битность 2048. Но учитывая значение результата R2 можно начать подозревать, что химическая структура молекулы не является очень хорошим признаком для определения pEC50. 
Но далее мы проверим несколько различных регрисионных моделей, а так же проведем оптимизацию их гиперпараметров

In [ ]:
d = np.ones(shape=(2048,))
for fp in X_test['fp2,2048']:
    d = np.vstack((d,np.array(fp)))
d = np.delete(d,0,0)        
Test = d
d = np.ones(shape=(2048,))
for fp in X_train['fp2,2048']:
    d = np.vstack((d,np.array(fp)))
d = np.delete(d,0,0)     
Train = d


# Проведем оптимизацию гиперпараметров для RFR

Сначала воспользуемся RandomizedSearchCV а потом уже GridSearchem

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rfr = RandomForestRegressor()
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(rfr, 
                        param_dist, 
                        n_iter = 100, 
                        cv = 5, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(Train, y_train)
rs.best_params_

После чего строю графики для каждого параметра, где можно будет визуально увидеть лучшие значения

In [ ]:
rs_df = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_df = rs_df.drop([
            'mean_fit_time', 
            'std_fit_time', 
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score', 
            'std_test_score'],
            axis=1)
fig, axs = plt.subplots(ncols=3, nrows=2)
sns.set(style="whitegrid", color_codes=True, font_scale = 2)
fig.set_size_inches(30,25)
sns.barplot( data=rs_df,x='param_n_estimators', y='mean_test_score', ax=axs[0,0], color='lightgrey')
axs[0,0].set_ylim([0,0.4])
axs[0,0].set_title(label = 'n_estimators', size=30, weight='bold')
sns.barplot(x='param_min_samples_split', y='mean_test_score', data=rs_df, ax=axs[0,1], color='coral')
axs[0,1].set_ylim([0,0.4])
axs[0,1].set_title(label = 'min_samples_split', size=30, weight='bold')
sns.barplot(x='param_min_samples_leaf', y='mean_test_score', data=rs_df, ax=axs[0,2], color='lightgreen')
axs[0,2].set_ylim([0,0.4])
axs[0,2].set_title(label = 'min_samples_leaf', size=30, weight='bold')
sns.barplot(x='param_max_features', y='mean_test_score', data=rs_df, ax=axs[1,0], color='wheat')
axs[1,0].set_ylim([0,0.4])
axs[1,0].set_title(label = 'max_features', size=30, weight='bold')
sns.barplot(x='param_max_depth', y='mean_test_score', data=rs_df, ax=axs[1,1], color='lightpink')
axs[1,1].set_ylim([0,0.4])
axs[1,1].set_title(label = 'max_depth', size=30, weight='bold')
sns.barplot(x='param_bootstrap',y='mean_test_score', data=rs_df, ax=axs[1,2], color='skyblue')
axs[1,2].set_ylim([0,0.4])
axs[1,2].set_title(label = 'bootstrap', size=30, weight='bold')
plt.show()


Далее уже из вручную выбранных значений, с помощью GridSearchCV находим наилучшую комбинацию гиперпараметров для нашей модели

In [ ]:
from sklearn.model_selection import GridSearchCV
n_estimators = [500,700]
min_samples_split = [7,23]
min_samples_leaf = [2,18]
max_features = ['sqrt']
max_depth = [6,13,15]
bootstrap = [False,True]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(rfr, param_grid, cv = 5, verbose = 1, n_jobs=-1)
gs.fit(Train, y_train)
rfc_3 = gs.best_estimator_
gs.best_params_

In [ ]:
y_pred = gs.best_estimator_.predict(Test)
print('R2 score:', r2_score(y_test,y_pred))
print('Mean squared error:',mean_squared_error(y_test,y_pred))
print('Mean absolute error:',mean_absolute_error(y_test,y_pred))
print('Mean absolute percentage error:',mean_absolute_percentage_error(y_test,y_pred))

Модель объясняет 32% дисперсии y, а так же в принципе дает не такой уж плохой результат (далеко не хороший, но не ужасный). При предсказании мы в среднем ошибаемся на 11 процентов.

# Попробуем понизить размерность отпечатков с помощью PCA

In [ ]:
from sklearn.decomposition import PCA
pca_test = PCA(n_components=2000)
pca_test.fit(Train)
sns.set(style='whitegrid')
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(linewidth=4, color='r', linestyle = '--', x=250, ymin=0, ymax=1)
display(plt.show())
evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)
pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.iloc[240:251])

Взяв 250 главных компонент мы объясняем 95% дисперсии данных, попробуем посмотреть на результаты предсказаний

In [ ]:
pca = PCA(n_components=250)
pca.fit(Train)
Train_pca = pca.transform(Train)
pca.fit(Test)
Test_pca = pca.transform(Test)

In [ ]:
n_estimators = [500,700]
min_samples_split = [7,23]
min_samples_leaf = [2,18]
max_features = ['sqrt']
max_depth = [6,13,15]
bootstrap = [False,True]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(rfr, param_grid, cv = 5, verbose = 1, n_jobs=-1)
gs.fit(Train_pca, y_train)
gs.best_params_

In [ ]:
y_pred = gs.best_estimator_.predict(Test_pca)
print('R2 score:', r2_score(y_test,y_pred))
print('Mean squared error:',mean_squared_error(y_test,y_pred))
print('Mean absolute error:',mean_absolute_error(y_test,y_pred))
print('Mean absolute percentage error:',mean_absolute_percentage_error(y_test,y_pred))

Все стало еще грустнее, но это неудивительно. Применение PCA на отпечатки не несет большого смысла в силу того, что уменьшая размерность путем проецирования на гиперплоскость нужной размерности мы теряем структуру отпечатков, где нужные нам фрагменты описывались единичками. Мы мало того, что теряем объясненную дисперсию, но еще и сильно меняем структуру фичей не в лучшую сторону

# Попробуем градиентный бустинг..

In [ ]:
reg = ensemble.GradientBoostingRegressor()
n_estimators = [500]
min_samples_split = [7]
min_samples_leaf = [10]
max_depth = [13]
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
gs = GridSearchCV(reg, param_grid, cv = 5, verbose = 1, n_jobs=-1)
gs.fit(Train, y_train)
gs.best_params_

In [ ]:
y_pred = gs.best_estimator_.predict(Test)
print('R2 score:', r2_score(y_test,y_pred))
print('Mean squared error:',mean_squared_error(y_test,y_pred))
print('Mean absolute error:',mean_absolute_error(y_test,y_pred))
print('Mean absolute percentage error:',mean_absolute_percentage_error(y_test,y_pred))

Градиентый бустинг для данной задачи показывает себя ощутимо хуже по R2, но чуть лучше по MAE

# Перестаем мучить отпечатки и воспользуемся дескрипторами RDKit

# Часть вторая

Из молфайлов получим набор всех дескрипторов

In [ ]:
df_desc = df_work[['SMILES','pEC50']]
df_desc = df_desc.reset_index()
df_desc

In [ ]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles] 
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    
    Mol_descriptors =[]
    for mol in mols:
        # add hydrogens to molecules
        mol=Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names 

In [ ]:
Mol_descriptors,desc_names = RDkit_descriptors(df_desc['SMILES'])

In [ ]:
df_with_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)
df_with_descriptors

In [ ]:
df_with_descriptors.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_with_descriptors)
scaled_df = pd.DataFrame(scaler.transform(df_with_descriptors), columns = df_with_descriptors.columns)
scaled_df['pEC50'] = df_desc['pEC50']
scaled_df

In [ ]:
scaled_df.isnull().any(axis=1).sum() # у нас есть 15 строк с пропусками

In [ ]:
scaled_df.isnull().sum().sum()#суммарно у нас пропущено 180 значениф

Найдем колонки с пропусками

In [ ]:
a = scaled_df.loc[:, scaled_df.isnull().any()]
a

Найдем строки среди данных колонок с пропусками

In [ ]:
a[a.isnull().any(axis=1)]

Если мы просто выбросим все данные строки, то мы потеряем 15 строк из 2858, что составляет половину процента от всей базы.
Мне кажется, что дозаполнять данный датасет не имеет смысла. Избавимся от строк с пустыми значениями

In [ ]:
scaled_df = scaled_df.dropna()
scaled_df

С помощью метода IsolationForest мы можем избавиться от того, что данная модель посчитает выбросами, что может быть полезно для нас

In [ ]:
from sklearn.ensemble import IsolationForest
data_imputed=scaled_df
iforest = IsolationForest(n_estimators = 500, random_state=0,
    max_samples = "auto", 
    contamination= 0.05 #зададим сами процент потенциальных аномалий в БД
                          #Иначе алгоритм попытается определить это самостоятельно, иногда удаляя слишком много данных
    )
iforest_fit = iforest.fit(data_imputed)
predictions = iforest_fit.predict(data_imputed)

data_imputed['is_anomaly_prediction'] = predictions
data_imputed = data_imputed[data_imputed.is_anomaly_prediction != -1]
data_imputed = data_imputed.drop(columns = ['is_anomaly_prediction'])
data_imputed.describe()

У нас осталось 2700 сэмплов из 2858 исходных.

# Посмотрим на кореляцию наших признаков

In [ ]:
sp_corr_mat = data_imputed.corr(method = 'spearman')
sp_corr_mat

In [ ]:
fig = plt.figure(figsize=(5, 3))
plt.title("Spearman Correlation Matrix")
mask = np.triu(np.ones_like(sp_corr_mat, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(sp_corr_mat,
           mask = mask,
           cmap="Blues", annot=True, center=0, fmt='.2f')


Выше мы можем на глаз видеть много различных признаков, которые сильно коррелируют. Но при таком количестве признаков на глаз найти все пары и убрать ненужные фичи практически невозможно

Поэтому воспользуемся функцией:
с ее помощью мы можем получить список списков пар всех фичей, корреляция которых превышает 0.6

In [ ]:
def print_highly_correlated(df, features, threshold=0.6):
    """Prints highly correlated features pairs in the data frame (helpful for feature engineering)"""
    corr_df = df[features].corr(method = 'spearman') # get correlations
    correlated_features = np.where(np.abs(corr_df) > threshold) # select ones above the abs threshold
    correlated_features = [(corr_df.iloc[x,y], x, y) for x, y in zip(*correlated_features) if x != y and x < y] # avoid duplication
    s_corr_list = sorted(correlated_features, key=lambda x: -abs(x[0])) # sort by correlation value
    g = []
    if s_corr_list == []:
        print("There are no highly correlated features with correlation above", threshold)
    else:
        for v, i, j in s_corr_list:
            cols = df[features].columns
            g.append([corr_df.columns[j],corr_df.index[i]])
            print ("%s and %s = %.3f" % (corr_df.index[i], corr_df.columns[j], v))
    return(g)

In [ ]:
high_cor_pairs = print_highly_correlated(df=data_imputed, features=data_imputed.columns)


In [ ]:
len(high_cor_pairs)

Итого у нас есть 208 пар высокоскоррелированных признаков, сейчас нам нужно начать удалять признаки, но так чтобы не удалить лишние. Так как есть пересечения между различными признаками

In [ ]:
def print_highly_correlated2(df, features, threshold=0.6):
    """Prints highly correlated features pairs in the data frame (helpful for feature engineering)"""
    corr_df = df[features].corr(method = 'spearman') # get correlations
    correlated_features = np.where(np.abs(corr_df) > threshold) # select ones above the abs threshold
    correlated_features = [(corr_df.iloc[x,y], x, y) for x, y in zip(*correlated_features) if x != y and x < y] # avoid duplication
    s_corr_list = sorted(correlated_features, key=lambda x: -abs(x[0])) # sort by correlation value
    g = []
    if s_corr_list == []:
        print("There are no highly correlated features with correlation above", threshold)
    else:
        for v, i, j in s_corr_list:
            cols = df[features].columns
            g.append([corr_df.columns[j],corr_df.index[i]])
    return(g)

In [ ]:
df_drop = data_imputed.drop(high_cor_pairs[0][0],axis=1)
df_drop

In [ ]:
while high_cor_pairs != []:
    high_cor_pairs = print_highly_correlated2(df=df_drop, features=df_drop.columns)
    if high_cor_pairs == []:
        break
    df_drop = df_drop.drop(high_cor_pairs[0][0],axis=1)


Теперь в нашем датасете нет ни одной пары признаков корреляция спирмана для которых >=0.6

In [ ]:
df_drop

In [ ]:
sp_corr_mat = df_drop.corr(method = 'spearman')
fig = plt.figure(figsize=(5, 3))
plt.title("Spearman Correlation Matrix")
mask = np.triu(np.ones_like(sp_corr_mat, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(sp_corr_mat,
           mask = mask,
           cmap="Blues", annot=True, center=0, fmt='.2f')


Как видим теперь максимум на графике соответсвует 0.6, чего мы и хотели добиться :)
цифра 0.6 появилась постфактум, потому что изначально я брал значение в 0.85, но разница между моделями была еще меньше..

Таким образом мы понизили размерность с 211 до 114

# Battle двух датасетов:

Дата сет со всеми посчитанными дескрипторами RDKit (**df1**) vs Очищенный от лишних фичей (по критерию корреляции Спирмана) (**df2**) 

In [ ]:
df1 = data_imputed
df1

In [ ]:
df2 = data_imputed[df_drop.columns]
df2

In [ ]:
X = df1.drop(['pEC50'],axis=1)
y = df1['pEC50']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=13)

In [ ]:
X_traindf2 = X_train[df_drop.drop(['pEC50'],axis=1).columns]
X_testdf2 =  X_test[df_drop.drop(['pEC50'],axis=1).columns]

У нас готовы одинаковые сэмплы, X_train (df1) и X_traindf2 (df2), теперь мы можем объективно сравнивать эффективность моделей основанных на наших датасетах

In [ ]:
X_traindf2

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rfr = RandomForestRegressor()
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(rfr, 
                        param_dist, 
                        n_iter = 100, 
                        cv = 3, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(X_traindf2, y_train)
rs.best_params_

In [ ]:
rs_df = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_df = rs_df.drop([
            'mean_fit_time', 
            'std_fit_time', 
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score', 
            'std_test_score'],
            axis=1)
fig, axs = plt.subplots(ncols=3, nrows=2)
sns.set(style="whitegrid", color_codes=True, font_scale = 2)
fig.set_size_inches(30,25)
sns.barplot( data=rs_df,x='param_n_estimators', y='mean_test_score', ax=axs[0,0], color='lightgrey')
axs[0,0].set_ylim([0,0.4])
axs[0,0].set_title(label = 'n_estimators', size=30, weight='bold')
sns.barplot(x='param_min_samples_split', y='mean_test_score', data=rs_df, ax=axs[0,1], color='coral')
axs[0,1].set_ylim([0,0.4])
axs[0,1].set_title(label = 'min_samples_split', size=30, weight='bold')
sns.barplot(x='param_min_samples_leaf', y='mean_test_score', data=rs_df, ax=axs[0,2], color='lightgreen')
axs[0,2].set_ylim([0,0.4])
axs[0,2].set_title(label = 'min_samples_leaf', size=30, weight='bold')
sns.barplot(x='param_max_features', y='mean_test_score', data=rs_df, ax=axs[1,0], color='wheat')
axs[1,0].set_ylim([0,0.4])
axs[1,0].set_title(label = 'max_features', size=30, weight='bold')
sns.barplot(x='param_max_depth', y='mean_test_score', data=rs_df, ax=axs[1,1], color='lightpink')
axs[1,1].set_ylim([0,0.4])
axs[1,1].set_title(label = 'max_depth', size=30, weight='bold')
sns.barplot(x='param_bootstrap',y='mean_test_score', data=rs_df, ax=axs[1,2], color='skyblue')
axs[1,2].set_ylim([0,0.4])
axs[1,2].set_title(label = 'bootstrap', size=30, weight='bold')
plt.show()


In [ ]:
n_estimators = [500,700]
min_samples_split = [23,44]
min_samples_leaf = [2,18]
max_features = ['sqrt']
max_depth = [7,13,15]
bootstrap = [True]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(rfr, param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(X_traindf2, y_train)
rfc_3 = gs.best_estimator_
gs.best_params_

In [ ]:
y_pred = gs.best_estimator_.predict(X_testdf2)
print('R2 score:', r2_score(y_test,y_pred))
print('Mean squared error:',mean_squared_error(y_test,y_pred))
print('Mean absolute error:',mean_absolute_error(y_test,y_pred))
print('Mean absolute percentage error:',mean_absolute_percentage_error(y_test,y_pred))

Результаты оказались не настолько впечатляющими, как я ожидал

In [ ]:
rfc_3.fit(X_train, y_train)
y_pred = rfc_3.predict(X_test)
print('R2 score:', r2_score(y_test,y_pred))
print('Mean squared error:',mean_squared_error(y_test,y_pred))
print('Mean absolute error:',mean_absolute_error(y_test,y_pred))
print('Mean absolute percentage error:',mean_absolute_percentage_error(y_test,y_pred))

Разница оказалась минимальна для RFR.......
Да, в случае df2, где размерность ниже результат немного хуже, но это совсем минимальная разница, нооооо мы уменьшили размерность ощутимо! Не потеряв качества, это уже победа

Посмотрим градиентный бустинг.. (я не перебираю большое количество гиперпараметров в силу того, что мой комп итак взрывается от него..)

In [ ]:
reg = ensemble.GradientBoostingRegressor()
n_estimators = [500]
min_samples_split = [13]
min_samples_leaf = [10]
max_depth = [13]
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
gs = GridSearchCV(reg, param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(X_traindf2, y_train)
gbr = gs.best_estimator_
gs.best_params_

In [ ]:
y_pred = gs.best_estimator_.predict(X_testdf2)
print('R2 score:', r2_score(y_test,y_pred))
print('Mean squared error:',mean_squared_error(y_test,y_pred))
print('Mean absolute error:',mean_absolute_error(y_test,y_pred))
print('Mean absolute percentage error:',mean_absolute_percentage_error(y_test,y_pred))

In [ ]:
gbr.fit(X_train, y_train)
y_pred = rfc_3.predict(X_test)
print('R2 score:', r2_score(y_test,y_pred))
print('Mean squared error:',mean_squared_error(y_test,y_pred))
print('Mean absolute error:',mean_absolute_error(y_test,y_pred))
print('Mean absolute percentage error:',mean_absolute_percentage_error(y_test,y_pred))

Градиентный бустинг дал другую ситуацию... но с гораздо большей разницей между сетами


# Попробуем понизить размерность с помощью PCA

In [ ]:
from sklearn.decomposition import PCA
pca_test = PCA(n_components=210)
pca_test.fit(X_train)
sns.set(style='whitegrid')
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(linewidth=4, color='r', linestyle = '--', x=53, ymin=0, ymax=1)
display(plt.show())
evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)
pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.iloc[50:60])

Взяв 53 главных компонент мы объясняем 95% дисперсии данных, попробуем посмотреть на результаты предсказаний

In [ ]:
pca = PCA(n_components=53)
pca.fit(X_train)
Train_pca = pca.transform(X_train)
pca.fit(X_test)
Test_pca = pca.transform(X_test)

In [ ]:
rfc_3.fit(Train_pca,y_train)
y_pred = rfc_3.predict(Test_pca)
print('R2 score:', r2_score(y_test,y_pred))
print('Mean squared error:',mean_squared_error(y_test,y_pred))
print('Mean absolute error:',mean_absolute_error(y_test,y_pred))
print('Mean absolute percentage error:',mean_absolute_percentage_error(y_test,y_pred))

Как видно, использование метода главных компонент на датасете со всеми дескрипторами дает ощутимо худший результат, что логично, ведь у нас крайне много высокоскоррелированных признаков, которые никуда не делись, а лишь проецировались на гиперплоскость..

Посмотрим результат на исходных данных из датасета, да мы разобьем данные най трейн и тест другим образом, результат нельзя будет сравнивать напрямую, но оценить порядок ошибки мы сможем  
Здесь я энкодю категориальную переменную, чтобы она правильно воспринималась нашими моделями

In [ ]:
old = filtered_df.drop(['Assay Type','EC50 (nM)','SMILES'],axis=1)
df_encoded = pd.get_dummies(old, columns=['Target Name'], dtype = int)
y = df_encoded['pEC50']
X = df_encoded.drop('pEC50',axis=1)
df_encoded

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=13)

In [ ]:
n_estimators = [500,700]
min_samples_split = [23,44]
min_samples_leaf = [2,18]
max_features = ['sqrt']
max_depth = [7,13,15]
bootstrap = [False,True]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(rfr, param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(X_train, y_train)
rfc_3 = gs.best_estimator_
gs.best_params_

In [ ]:
y_pred = gs.best_estimator_.predict(X_test)
print('R2 score:', r2_score(y_test,y_pred))
print('Mean squared error:',mean_squared_error(y_test,y_pred))
print('Mean absolute error:',mean_absolute_error(y_test,y_pred))
print('Mean absolute percentage error:',mean_absolute_percentage_error(y_test,y_pred))

# Итого мы имеем следующие результаты для задачи регрессии:

В случае использования отпечатков моргана радиуса 2, битности 2048 и использования **RandomForestRegressor** в качестве модели, при оптимизации гиперпараметров мы получаем: (в схожих исследованиях в основном ориентируется на значение MAE, как ключевое)   
**R2 score: 0.3154795876046378  
Mean absolute error: 0.7552310235089378**

Если мы используем все то же самое, но дополнительно пользуемся **PCA** для понижения размерности мы получаем результаты ощутимо хуже:  
**R2 score: 0.060187806504677366  
Mean absolute error: 0.9014812592919748**

**GradientBoostingRegressor** для данной задачи показывает себя хуже по R2, но чуть лучше по MAE  
**R2 score: 0.27147601901556573  
Mean absolute error: 0.7475003831849716**


И казалось, что отсюда хочется сделать вывод, что задачу QSAR для данного датасета не получится решить эффективно, и следущий логичный шаг это посмотреть на результаты использования дескрипторов RDKit.  
Таким образом мы получили два датасета, один в котором содержатся все дискрипторы посчитанные RDKit, без какой либо доп обработки, кроме удаления NaN. Над вторым сетом уже проведена большая работа, в нем был проведен анализ корреляций признаков Спирмана и исходя их этого были удалены признаки таким образом, чтобы не осталось пар, для который корреляция спирмана превосходила бы **0.6**.  

Результаты **RFR** для датасета с дескрипторами без обработки  
**R2 score: 0.37442611209050225  
Mean absolute error: 0.6947616846466108**  

Результаты **GBR** для датасета с дескрипторами без обработки  
**R2 score: 0.37442611209050225  
Mean absolute error: 0.6947616846466108**  

Результаты **RFR** для датасета с обработкой высокоскоррелированных признаков:  
**R2 score: 0.36710239313100945  
Mean absolute error: 0.7059629466610295**  

Результаты **GBR** для датасета с обработкой высокоскоррелированных признаков:   
**R2 score: 0.21537929036797976  
Mean absolute error: 0.7338091826656238**  

Результаты **RFR** для датасета со всеми дескрипторами, но с использованием PCA:     
**R2 score: 0.02199217071670001  
Mean absolute error: 0.9114822618885962**  

Также можно посмотреть на результаты для модели, обученной на заранее отобранных авторами датасета признаков:  
**RFR**:  
**R2 score: 0.3199306108855291  
Mean absolute error: 0.7261527544544696**  

Таким образом с помощью дескрипторов RDKita мы смогли добиться ощутимо лучших результатов как по R2 так и по MAE
Оптимальным методом для решения данной проблемы, это использовать дескрипторы RDKita с последующим удалением ненужных высоскоррелированных фичей и в качестве модели использовать RandomForestRegressor с последующей оптимизацией гиперпараметров.

В интернете я нашел проект чувака, который на этом датасете решал схожую задачу, и невероятно, но у меня результат по MAE лучше на 0.1:)  

итого часть про регрессию все...

# Часть 3: Задача Классификации с помощью отпечатков

С помощью отпечатков моргана в качестве фичей создадим модель классификации одного из 5 типов допаминовых рецепторов (Target Name)

In [ ]:
class_df = filtered_df[['SMILES','Target Name']]
class_df

На всякий случай канонизируем смайлз

In [ ]:
class_df['SMILES'] = class_df['SMILES'].apply(lambda x: Chem.CanonSmiles(x))

Добавим столбец с мол-файлами

In [ ]:
PandasTools.AddMoleculeColumnToFrame(class_df, smilesCol='SMILES', molCol='Molecule')

In [ ]:
class_df

Добавим отпечатки пальцев r = 2, битность 2048

In [ ]:
def fpgen(x,n,size):
    mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=n,fpSize=size,includeChirality=True)
    e = mfpgen.GetFingerprint(x)#получаем FP
    return (e) 

In [ ]:
r = [2]
bit = [2048]
for i in r:
    for b in bit:
        class_df[f'fp{i},{b}'] = class_df['Molecule'].apply(lambda x: fpgen(x,i,b))
class_df

In [ ]:
class_df['Target Name'].value_counts()

Из распределения классов выше видно, что D5 слишком мало. Проще его убрать и сосредоточиться на предсказании оставшихся

In [ ]:
class_df = class_df[class_df['Target Name'] !='Dopamine D5 receptor']

In [ ]:
class_df

Разбиваем сет на трейн и тест с самого начала

In [ ]:
X = class_df.drop(['SMILES','Target Name','Molecule'],axis=1)
y = class_df['Target Name']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=13, stratify=y)

In [ ]:
d = np.ones(shape=(2048,))
for fp in X_test['fp2,2048']:
    d = np.vstack((d,np.array(fp)))
d = np.delete(d,0,0)        
Test = d
d = np.ones(shape=(2048,))
for fp in X_train['fp2,2048']:
    d = np.vstack((d,np.array(fp)))
d = np.delete(d,0,0)     
Train = d

У нас готов датасет с отпечатками радиуса =2 и битностью 2048 в качестве фичей

# Проведем оптимизацию гиперпараметров для RFC

Сначала воспользуемся RandomizedSearchCV а потом уже GridSearchem

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(rfc, 
                        param_dist, 
                        n_iter = 100, 
                        cv = 5, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(Train, y_train)
rs.best_params_

После чего строю графики для каждого параметра, где можно будет визуально увидеть лучшие значения

In [ ]:
rs_df = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_df = rs_df.drop([
            'mean_fit_time', 
            'std_fit_time', 
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score', 
            'std_test_score'],
            axis=1)
fig, axs = plt.subplots(ncols=3, nrows=2)
sns.set(style="whitegrid", color_codes=True, font_scale = 2)
fig.set_size_inches(30,25)
sns.barplot( data=rs_df,x='param_n_estimators', y='mean_test_score', ax=axs[0,0], color='lightgrey')
axs[0,0].set_ylim([0,0.8])
axs[0,0].set_title(label = 'n_estimators', size=30, weight='bold')
sns.barplot(x='param_min_samples_split', y='mean_test_score', data=rs_df, ax=axs[0,1], color='coral')
axs[0,1].set_ylim([0,0.8])
axs[0,1].set_title(label = 'min_samples_split', size=30, weight='bold')
sns.barplot(x='param_min_samples_leaf', y='mean_test_score', data=rs_df, ax=axs[0,2], color='lightgreen')
axs[0,2].set_ylim([0,0.8])
axs[0,2].set_title(label = 'min_samples_leaf', size=30, weight='bold')
sns.barplot(x='param_max_features', y='mean_test_score', data=rs_df, ax=axs[1,0], color='wheat')
axs[1,0].set_ylim([0,0.8])
axs[1,0].set_title(label = 'max_features', size=30, weight='bold')
sns.barplot(x='param_max_depth', y='mean_test_score', data=rs_df, ax=axs[1,1], color='lightpink')
axs[1,1].set_ylim([0,0.8])
axs[1,1].set_title(label = 'max_depth', size=30, weight='bold')
sns.barplot(x='param_bootstrap',y='mean_test_score', data=rs_df, ax=axs[1,2], color='skyblue')
axs[1,2].set_ylim([0,0.8])
axs[1,2].set_title(label = 'bootstrap', size=30, weight='bold')
plt.show()


Далее уже из вручную выбранных значений, с помощью GridSearchCV находим наилучшую комбинацию гиперпараметров для нашей модели

In [ ]:
from sklearn.model_selection import GridSearchCV
n_estimators = [500,700]
min_samples_split = [2,23]
min_samples_leaf = [2,18]
max_features = ['sqrt']
max_depth = [6,15]
bootstrap = [True]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(rfc, param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(Train, y_train)
rfc_3 = gs.best_estimator_
gs.best_params_

In [ ]:
y_pred = gs.best_estimator_.predict(Test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Результаты неплохие, мы в 0.76 случаев определяем правильный класс, при этом имея достоный F1-Score почти для всех классов

# Попробуем понизить размерность с помощью PCA

In [ ]:
from sklearn.decomposition import PCA
pca_test = PCA(n_components=1977)
pca_test.fit(Train)
sns.set(style='whitegrid')
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(linewidth=4, color='r', linestyle = '--', x=246, ymin=0, ymax=1)
display(plt.show())
evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)
pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.iloc[240:250])

Взяв 246 главных компонент мы объясняем 95% дисперсии данных, попробуем посмотреть на результаты предсказаний

In [ ]:
pca = PCA(n_components=246)
pca.fit(Train)
Train_pca = pca.transform(Train)
pca.fit(Test)
Test_pca = pca.transform(Test)

In [ ]:
rfc_3.fit(Train_pca,y_train)
y_pred = gs.best_estimator_.predict(Test_pca)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Что было понятно с самого начала, применение PCA для отпечатков не приводит к хорошим результам, так поступать не стоит!

# Проверим GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = ensemble.GradientBoostingClassifier(random_state = 0)
n_estimators = [500]
min_samples_split = [6]
min_samples_leaf = [10]
max_depth = [10]
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
gs = GridSearchCV(gbc, param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(Train, y_train)
gbr = gs.best_estimator_
gs.best_params_

In [ ]:
y_pred = gs.best_estimator_.predict(Test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Градиентый бустинг оказался чуть менее точным, но зато с лучшим средним F1-score...

# 4 Часть: классификация с помощью дескрипторов RDKit

In [ ]:
df_desc2 = filtered_df['Target Name']
df_desc2 = df_desc2.reset_index()
df_desc2

In [ ]:
from sklearn.preprocessing import LabelEncoder
df_desc2 = df_desc2[df_desc2['Target Name'] !='Dopamine D5 receptor']
le = LabelEncoder()
le.fit(df_desc2['Target Name'])
df_desc2['Target Name'] = le.transform(df_desc2['Target Name'])
df_desc2['Target Name'].value_counts()

In [ ]:
## from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_with_descriptors)
scaled_df2 = pd.DataFrame(scaler.transform(df_with_descriptors), columns = df_with_descriptors.columns)
scaled_df2['Target Name'] = df_desc2['Target Name']
scaled_df2 = scaled_df2.dropna()
scaled_df2

In [ ]:
from sklearn.ensemble import IsolationForest
data_imputed=scaled_df2
iforest = IsolationForest(n_estimators = 500, random_state=0,
    max_samples = "auto", 
    contamination= 0.05 #зададим сами процент потенциальных аномалий в БД
                          #Иначе алгоритм попытается определить это самостоятельно, иногда удаляя слишком много данных
    )
iforest_fit = iforest.fit(data_imputed)
predictions = iforest_fit.predict(data_imputed)

data_imputed['is_anomaly_prediction'] = predictions
data_imputed = data_imputed[data_imputed.is_anomaly_prediction != -1]
data_imputed = data_imputed.drop(columns = ['is_anomaly_prediction'])
data_imputed.describe()

# Посмотрим на кореляцию наших признаков

In [ ]:
sp_corr_mat = data_imputed.corr(method = 'spearman')
fig = plt.figure(figsize=(5, 3))
plt.title("Spearman Correlation Matrix")
mask = np.triu(np.ones_like(sp_corr_mat, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(sp_corr_mat,
           mask = mask,
           cmap="Blues", annot=True, center=0, fmt='.2f')


Выше мы можем на глаз видеть много различных признаков, которые сильно коррелируют. Но при таком количестве признаков на глаз найти все пары и убрать ненужные фичи практически невозможно

Поэтому воспользуемся функцией:
с ее помощью мы можем получить список списков пар всех фичей, корреляция которых превышает 0.6

In [ ]:
def print_highly_correlated(df, features, threshold=0.6):
    """Prints highly correlated features pairs in the data frame (helpful for feature engineering)"""
    corr_df = df[features].corr(method = 'spearman') # get correlations
    correlated_features = np.where(np.abs(corr_df) > threshold) # select ones above the abs threshold
    correlated_features = [(corr_df.iloc[x,y], x, y) for x, y in zip(*correlated_features) if x != y and x < y] # avoid duplication
    s_corr_list = sorted(correlated_features, key=lambda x: -abs(x[0])) # sort by correlation value
    g = []
    if s_corr_list == []:
        print("There are no highly correlated features with correlation above", threshold)
    else:
        for v, i, j in s_corr_list:
            cols = df[features].columns
            g.append([corr_df.columns[j],corr_df.index[i]])
            print ("%s and %s = %.3f" % (corr_df.index[i], corr_df.columns[j], v))
    return(g)

In [ ]:
high_cor_pairs = print_highly_correlated(df=data_imputed, features=data_imputed.columns)

Итого у нас есть 700+ пар высокоскоррелированных признаков, сейчас нам нужно начать удалять признаки, но так чтобы не удалить лишние. Так как есть пересечения между различными признаками

In [ ]:
def print_highly_correlated2(df, features, threshold=0.6):
    """Prints highly correlated features pairs in the data frame (helpful for feature engineering)"""
    corr_df = df[features].corr(method = 'spearman') # get correlations
    correlated_features = np.where(np.abs(corr_df) > threshold) # select ones above the abs threshold
    correlated_features = [(corr_df.iloc[x,y], x, y) for x, y in zip(*correlated_features) if x != y and x < y] # avoid duplication
    s_corr_list = sorted(correlated_features, key=lambda x: -abs(x[0])) # sort by correlation value
    g = []
    if s_corr_list == []:
        print("There are no highly correlated features with correlation above", threshold)
    else:
        for v, i, j in s_corr_list:
            cols = df[features].columns
            g.append([corr_df.columns[j],corr_df.index[i]])
    return(g)

In [ ]:
df_drop = data_imputed.drop(high_cor_pairs[0][0],axis=1)
df_drop

In [ ]:
while high_cor_pairs != []:
    high_cor_pairs = print_highly_correlated2(df=df_drop, features=df_drop.columns)
    if high_cor_pairs == []:
        break
    df_drop = df_drop.drop(high_cor_pairs[0][0],axis=1)


Теперь в нашем датасете нет ни одной пары признаков корреляция спирмана для которых >=0.6

In [ ]:
df_drop

In [ ]:
sp_corr_mat = df_drop.corr(method = 'spearman')
fig = plt.figure(figsize=(5, 3))
plt.title("Spearman Correlation Matrix")
mask = np.triu(np.ones_like(sp_corr_mat, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(sp_corr_mat,
           mask = mask,
           cmap="Blues", annot=True, center=0, fmt='.2f')


Как видим теперь максимум на графике соответсвует 0.6, чего мы и хотели добиться :)
цифра 0.6 появилась постфактум, потому что изначально я брал значение в 0.85, но разница между моделями была еще меньше..

Таким образом мы понизили размерность с 200+ до 112

# Battle двух датасетов:

Дата сет со всеми посчитанными дескрипторами RDKit (**df1**) vs Очищенный от лишних фичей (по критерию корреляции Спирмана) (**df2**) 

In [ ]:
df1 = data_imputed
df1

In [ ]:
df2 = data_imputed[df_drop.columns]
df2

In [ ]:
X = df1.drop(['Target Name'],axis=1)
y = df1['Target Name']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=13)

In [ ]:
X_traindf2 = X_train[df_drop.drop(['Target Name'],axis=1).columns]
X_testdf2 =  X_test[df_drop.drop(['Target Name'],axis=1).columns]

# Проведем оптимизацию гиперпараметров для RFC для подготовиленного датасета


Сначала воспользуемся RandomizedSearchCV а потом уже GridSearchem

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(rfc, 
                        param_dist, 
                        n_iter = 100, 
                        cv = 5, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=0)
rs.fit(X_traindf2, y_train)
rs.best_params_

После чего строю графики для каждого параметра, где можно будет визуально увидеть лучшие значения

In [ ]:
rs_df = pd.DataFrame(rs.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
rs_df = rs_df.drop([
            'mean_fit_time', 
            'std_fit_time', 
            'mean_score_time',
            'std_score_time', 
            'params', 
            'split0_test_score', 
            'split1_test_score', 
            'split2_test_score', 
            'std_test_score'],
            axis=1)
fig, axs = plt.subplots(ncols=3, nrows=2)
sns.set(style="whitegrid", color_codes=True, font_scale = 2)
fig.set_size_inches(30,25)
sns.barplot( data=rs_df,x='param_n_estimators', y='mean_test_score', ax=axs[0,0], color='lightgrey')
axs[0,0].set_ylim([0,0.8])
axs[0,0].set_title(label = 'n_estimators', size=30, weight='bold')
sns.barplot(x='param_min_samples_split', y='mean_test_score', data=rs_df, ax=axs[0,1], color='coral')
axs[0,1].set_ylim([0,0.8])
axs[0,1].set_title(label = 'min_samples_split', size=30, weight='bold')
sns.barplot(x='param_min_samples_leaf', y='mean_test_score', data=rs_df, ax=axs[0,2], color='lightgreen')
axs[0,2].set_ylim([0,0.8])
axs[0,2].set_title(label = 'min_samples_leaf', size=30, weight='bold')
sns.barplot(x='param_max_features', y='mean_test_score', data=rs_df, ax=axs[1,0], color='wheat')
axs[1,0].set_ylim([0,0.8])
axs[1,0].set_title(label = 'max_features', size=30, weight='bold')
sns.barplot(x='param_max_depth', y='mean_test_score', data=rs_df, ax=axs[1,1], color='lightpink')
axs[1,1].set_ylim([0,0.8])
axs[1,1].set_title(label = 'max_depth', size=30, weight='bold')
sns.barplot(x='param_bootstrap',y='mean_test_score', data=rs_df, ax=axs[1,2], color='skyblue')
axs[1,2].set_ylim([0,0.8])
axs[1,2].set_title(label = 'bootstrap', size=30, weight='bold')
plt.show()


Далее уже из вручную выбранных значений, с помощью GridSearchCV находим наилучшую комбинацию гиперпараметров для нашей модели

In [ ]:
from sklearn.model_selection import GridSearchCV
n_estimators = [500,900]
min_samples_split = [2,23]
min_samples_leaf = [2,18]
max_features = ['sqrt']
max_depth = [7,12]
bootstrap = [True,False]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
gs = GridSearchCV(rfc, param_grid, cv = 3, verbose = 1, n_jobs=-1)
gs.fit(X_traindf2, y_train)
rfc_3 = gs.best_estimator_
gs.best_params_

In [ ]:
y_pred = gs.best_estimator_.predict(X_testdf2)
print(classification_report(y_test, y_pred))

Теперь посчитаем тоже самое для датасета без отфильтрованных фичей

In [ ]:
rfc_3.fit(X_train,y_train)
y_pred = rfc_3.predict(X_test)
print(classification_report(y_test, y_pred))

Разницы в выборе двух датасетов для нашей задачи классификации практически не оказалось, что отлично, ведь мы сократили размерность с 200+ до 112 с сохранением всех метрик!

# Попробуем понизить размерность с помощью PCA

In [ ]:
from sklearn.decomposition import PCA
pca_test = PCA(n_components=210)
pca_test.fit(X_train)
sns.set(style='whitegrid')
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(linewidth=4, color='r', linestyle = '--', x=53, ymin=0, ymax=1)
display(plt.show())
evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)
pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.iloc[50:55])

Взяв 53 главных компонент мы объясняем 95% дисперсии данных, но для сравнения с нашим методом понижения размерности путем удаление высокоскоррелированных признаков, возьмем нашу конечную размерность = 112

In [ ]:
pca = PCA(n_components=112)
pca.fit(X_train)
Train_pca = pca.transform(X_train)
pca.fit(X_test)
Test_pca = pca.transform(X_test)

In [ ]:
rfc_3.fit(Train_pca,y_train)
y_pred = gs.best_estimator_.predict(Test_pca)
print(classification_report(y_test, y_pred))

И снова PCA показывает результаты ощутимо хуже по всем метрикам! То есть наш метод понижения размерности гораздо эффективнее!

# Выводы по задаче классификации:

Используя отпечатки моргана r = 2, битность = 2048 и **RFC** в качестве модели мы получаем:  
-------------------------------**F1-score**  
Dopamine D1 receptor -          0.90      
Dopamine D2 receptor  -         0.79      
Dopamine D3 receptor   -      0.55       
Dopamine D4 receptor    -        0.83       
 ----------------------------**accuracy 0.76**       
 
Используя те же отпечатки и **RFC** в качестве модели, но с предварительным понижением размерности с помощью **PCA** мы получаем:  
-------------------------------**F1-score**  
Dopamine D1 receptor -          0.30      
Dopamine D2 receptor  -         0.67      
Dopamine D3 receptor   -      0.06       
Dopamine D4 receptor    -        0.00       
 ----------------------------**accuracy 0.52**      
 
Используя те же отпечатки и **GBC** в качестве модели мы получаем:  
-------------------------------**F1-score**  
Dopamine D1 receptor -          0.86      
Dopamine D2 receptor  -         0.74      
Dopamine D3 receptor   -      0.49       
Dopamine D4 receptor    -        0.85       
 ----------------------------**accuracy 0.72**      


Используя все дескрипторы полученные в **RDKit** - модель **RFC**:  
-------------------------------**F1-score**  
Dopamine D1 receptor - 0.86  
Dopamine D2 receptor - 0.76  
Dopamine D3 receptor - 0.56  
Dopamine D4 receptor - 0.83  
----------------------------**accuracy 0.74**  

Используя нескоррелированные дескрипторы полученные в **RDKit** - модель **RFC**:  
-------------------------------**F1-score**  
Dopamine D1 receptor - 0.85  
Dopamine D2 receptor - 0.76  
Dopamine D3 receptor - 0.57  
Dopamine D4 receptor - 0.82  
----------------------------**accuracy 0.74**  

Используя PCA в качестве алгоритма понижения размерности дескрипторов в **RDKit** - модель **RFC**:  
-------------------------------**F1-score**  
Dopamine D1 receptor - 0.45  
Dopamine D2 receptor - 0.69  
Dopamine D3 receptor - 0.20  
Dopamine D4 receptor - 0.06  
----------------------------**accuracy 0.56**  

Таким образом можно легко увидеть, что отпечатки моргана дают наилучший результат для предсказания класса лиганда дофаминового рецептора, исходя из чего можно сделать вывод, что химическая структура напрямую определяет класс лиганда, чего не было в случае решения задачи регрессии по предсказанию концентрации pEC50!   

Дескрипторы RDKit также дают схожую точность, наилучшим решением является использование не всех дескрипторов, а только те, которые не будут являться высокоскоррелированными друг с другом и моделью выбрать RFC

Еще один простой вывод, PCA не волшебная вещь, которая не позволяет магическим образом понизить размерность без потери качества модели, особенно когда мы говорим о понижении размерности таких систем, как отпечатки моргана... Во всех примерах в данной работе использование PCA приводило к сущетсвенному ухудшению работы модели. В то время как метод понижения размености путем анализа корреляции Спирмана для всех признаков с последующим удалением высокоскоррелированных, позволяет не просто понизить размерность, но даже не потерять эффективности работы модели! (а в случае больших объемов сетов, я уверен, что эффективность модели даже вырастет!) Так что Data-Preproccecing - наше все!

# The End